# Lektion 7 - Übung 3

Import von Python-Bibliotheken und notwendiger Code für diverse Funktionen

In [1]:
## imports
import math
import pandas as pd
from IPython.display import HTML, display


## Der folgende Code dient zur Implementierung der Funktionalität, damit Code ein- und ausgeblendet werden kann.
def toggle_code():
    toggle_code_js_html = '''
    <script>
    function code_toggle() {
        if ($('div.cell.code_cell.rendered.selected div.input').css('display')!='none'){
            $('div.cell.code_cell.rendered.selected div.input').hide();
        } else {
            $('div.cell.code_cell.rendered.selected div.input').show();
        }
    }
    </script>
    <form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Code ein-/ausblenden"></form>
    '''
    display(HTML(toggle_code_js_html))


toggle_code()

# Aufgabentext

An deutschen Hochschulen müssen unter bestimmten Umständen Studierende einen Sprachtest als Bestandteil des Zulassungsverfahrens absolvieren. Das Merkmal X "Punkte im Sprachtest" ist normalverteilt. Eine Stichprobenerhebung ergibt die folgenden Ergebnisse:  

66, 38, 73, 64, 85, 58, 32, 42, 77 und 45.  

Bestimmen Sie zunächst die Punktschätzer für den Erwartungswert und die Standardabweichung des Merkmals in der Grundgesamtheit.  Ermitteln Sie darauf aufbauend das 95%-Konfidenzintervall für den Erwartungswert in der Grundgesamtheit.  
Erläutern Sie anhand dessen den Unterschied der beiden Schätzverfahren.

# Lösung

**1. Festlegung der Konfidenzwahrscheinlichkeit 1 - $\alpha$**

Die Konfidenzwahrscheinlichkeit beträgt 1 - $\alpha$ = 0.95, d.h. $\alpha$ = 0.05

**2. Bestimmung der Anzahl der Freiheitsgrade und Ablesen des t-Werts**

Die Freiheitsgrade ergeben sich durch n - 1 = 10 - 1 = 9.  

Der t-Wert für 9 Freiheitsgrade und einer Konfidenzwahrscheinlichkeit von 95% beträgt 2,262.

In [2]:
import math
from scipy.stats import t

In [3]:
# Bei einem zweiseitigen Konfidenzintervall verwenden wir 1 - alpha / 2 = 1 - 0.05 / 2 = 1 - 0.025 = 0.975
t_value = t.ppf(0.975, 9)
print(t_value)

2.2621571627409915


**3.1 Punktschätzer für Erwartungswert $\hat{\mu}$**
$$ \hat{\mu} = \bar{x} = \frac{1}{n} \sum_{t=1}^n x_t $$

In [4]:
n = 10
x_quer = (66 + 38 + 73 + 64 + 85 + 58 + 32 + 42 + 77 + 45) / n
print(x_quer)

58.0


**3.2 Punktschätzer für Varianz $\sigma^2$ bzw. Standardabweichung $\sigma$**  

$$ \hat{\sigma}^2 = s^2 = \frac{1}{n - 1} \sum_{t=1}^n (x_t - \bar{x})^2 $$  
$$ \hat{\sigma} = s = \sqrt{s^2} $$

In [5]:
s_2 = ((66 - x_quer)**2 + (38 - x_quer)**2 + (73 - x_quer)**2 + (64 - x_quer)**2 + (85 - x_quer)**2 + 
       (58 - x_quer)**2 + (32 - x_quer)**2 + (42 - x_quer)**2 + (77 - x_quer)**2 + (45 - x_quer)**2) / (n - 1)
s = math.sqrt(s_2)
print(s)

18.0


**4. Berechnung der geschätzten Standardabweichung des Stichprobenmittelwerts $\hat{\sigma}_\bar{x} = \frac{s}{\sqrt{n}}$**

In [6]:
standard_error = s / math.sqrt(n)
print(standard_error)

5.692099788303082


**5. Berechnung des Stichprobenfehlers $\delta = t_{1-\alpha} \hat{\sigma}_\bar{x}$**

In [7]:
sample_error = t_value * standard_error
print(sample_error)

12.8764243071463


**6. Angabe des Konfidenzintervalls $KI = [\bar{x} - t_{1-\alpha} \hat{\sigma}_\bar{x} ;  \bar{x} + t_{1-\alpha} \hat{\sigma}_\bar{x}] $**

In [8]:
lower_bound = x_quer - sample_error
upper_bound = x_quer + sample_error
print(lower_bound.round(2), upper_bound.round(2))

45.12 70.88


# Interaktive Grafik
## Veränderung des Konfidenzintervalls bei Veränderung der Parameter

In [9]:
import math
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import t, norm
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual


%matplotlib inline


def make_plot(konfp=0.95, sigma=18, n=10):

    loc = 58
    
    alpha = 1 - konfp
    z_value = t.ppf(1 - alpha / 2, n - 1)
    standard_error = sigma / math.sqrt(n)
    
    lower_bound = loc - z_value * standard_error
    upper_bound = loc + z_value * standard_error
    
    x_values = np.linspace(lower_bound, upper_bound, 100)
    y_values = [1 for i in x_values]

    fig, ax = plt.subplots(figsize=(14, 2))
    ax.plot(x_values, y_values, linewidth=4)

    ## x-Achse
    ##
    plt.xlim(0, 100)
    ax.set_xticks(np.arange(0, 100, 5))

    ## y-Achse
    ##
    plt.ylim(0.9, 1.1)
    ax.set_yticks([])
    ax.set_yticklabels([])
    #ax.set_ylabel('$P(x)$', size=14)

    ## Sonstiges
    ##
    ax.set_title(r'Konfidenzintervall mit $\bar{x}=58$,' 
                 + r' $\sigma=${}, Konfidenzwahrscheinlichkeit {}, Stichprobengröße {}'.format(sigma, konfp, n), 
                 size=16)
    ax.grid(True, axis='x')

    ## Text für Untergrenze
    plt.annotate(
        '{:.2f}'.format(lower_bound),
        xy=(min(lower_bound, 54.3), 1.01),
        ha='center',
        size=14
    )

    ## Text für Obergrenze
    plt.annotate(
        '{:.2f}'.format(upper_bound),
        xy=(max(upper_bound, 61,6), 1.01),
        ha='center',
        size=14
    )

toggle_code()

In [10]:
interactive(
    make_plot, 
    konfp=widgets.FloatSlider(value=0.95, min=0.9, max=0.99, step=0.01, continuous_update=False, description='Konfidenzwahrscheinlichkeit'),
    sigma=widgets.IntSlider(value=18, min=1, max=30, step=1, continuous_update=False, description='Sigma'),
    n=widgets.IntSlider(value=10, min=5, max=20, step=1, continuous_update=False, description='Stichprobenumfang')
)

interactive(children=(FloatSlider(value=0.95, continuous_update=False, description='Konfidenzwahrscheinlichkei…

# Interaktive Grafik
## Vergleich der t-Verteilung mit der Standardnormalverteilung in Abhängigkeit der Freiheitsgrade

Liegt eine Stichprobe vor und die Standardabweichung der Grundgesamtheit muß geschätzt werden, dann wird die t-Verteilung zur Ermittlung des Standardfehlers verwendet.  
Ist die Standardabweichung der Grundgesamtheit bekannt, so kann die Normalverteilung verwendet werden.  
Ab einer Stichprobengröße von n = 30 nähert sich die t-Verteilung der Standardnormalverteilung an.

In [11]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import t
from scipy.stats import norm
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual


%matplotlib inline


def make_plot(df=5):
    pass

    x_values = np.linspace(-5, 5, 300)
    y_values_norm = [norm.pdf(i, loc=0, scale=1) for i in x_values]
    y_values_t = [t.pdf(i, df, loc=0, scale=1) for i in x_values]

    fig, ax = plt.subplots(figsize=(14, 8))

    ax.plot(x_values, y_values_norm, label="Normalverteilung")
    ax.plot(x_values, y_values_t, label="t-Verteilung, Freiheitsgrade {}".format(df))

    ax.set_title('Vergleich Dichtefunktion der Standard-Normalverteilung und der t-Verteilung', size=16)
    #ax.set_xlabel('$x$', size=14)
    #ax.set_ylabel('$P(x)$', size=14)

    ax.grid(True, axis='both')

    #ax.set_xticks(np.linspace(-3, 3, 13))
    #ax.set_xticklabels(x_labels)

    #ax.set_yticks(np.linspace(0.0, 1.0, 11))

    ## Wert neben Linie schreiben
    #for x, y in zip(x_values, y_values):
    #    ax.annotate(str(y), xy=(x, y - 0.04), size=14)

    #major_ticks = np.arange(0, 1.1, 0.1)    
    #ax.set_yticks(major_ticks)

    plt.legend()

toggle_code()

In [12]:
interactive(
    make_plot, 
    df=widgets.IntSlider(value=5, min=5, max=50, step=1, continuous_update=False, description='Freiheitsgrade'))

interactive(children=(IntSlider(value=5, continuous_update=False, description='Freiheitsgrade', max=50, min=5)…